In [21]:
import pandas as pd
from os import path
import os

#DATA_IN = "../../data/raw/semi-raw_data.csv"
DATA_IN = "../../data/raw/Jefferson_County_KY_Signalized_Intersections.csv"
CSV_OUT = "../../data/clean/Jefferson_County_KY_Signalized_Intersections.csv"
assert path.exists(DATA_IN)

df=pd.read_csv(DATA_IN)

len(df)

1091

In [2]:
df.columns



Index(['X', 'Y', 'OBJECTID', 'UNITID', 'SIGID', 'MAINSTREET', 'CROSSSTREET',
       'OWNER2', 'ROUTE', 'MILEPOINT', 'DESCRIPTION', 'TIMES', 'OWNER', 'TYPE',
       'INTID', 'ATMSID'],
      dtype='object')

In [3]:
keepers = ['X', 'Y', 'MAINSTREET', 'CROSSSTREET', 'DESCRIPTION', 'INTID', 'OBJECTID']
dropping = set(df.columns).difference(keepers)

renames = {'X':"longitude",
        'Y':'latitude',
        'MAINSTREET':'main_street', 
        'CROSSSTREET':'cross_street', 
        'DESCRIPTION':'description', 
        'INTID':'intid', 
        'OBJECTID':'objectid'}

dropped = df.drop(dropping, axis=1)
renamed = dropped.rename(renames, axis=1)

renamed.intid = renamed.intid.str.strip()


In [4]:

int_id_counts = renamed.intid.value_counts()
repeat_ids = int_id_counts[int_id_counts > 1]
id_2 = '2'
id_not2 = repeat_ids.index[1:]

discard = list()
for id in id_not2:
    D = renamed[renamed.intid == id]
    discard.extend(D.index[1:])
discard

filtered = renamed.drop(discard)
filtered.intid.value_counts()

intid
2                23
1604203842663     1
244589126D886     1
2785299966026     1
2785199966026     1
                 ..
375906564154      1
418544927132      1
436629017132      1
508029305629      1
1217075539999     1
Name: count, Length: 1048, dtype: int64

In [11]:
#assert not bool(set(filtered.objectid) & set(filtered.intid))
id2 = filtered[filtered.intid == '2']
clean = filtered.copy()
clean.loc[id2.index, 'intid'] = filtered.objectid
clean = clean.set_index("intid")

### Write out clean data to CSV

Run all to write file
Late cell does the writing step

In [22]:
cleaned = clean[['main_street', 'cross_street', 'longitude', 'latitude', 'description']]
len(cleaned)

1070

In [ ]:

cleaned.to_csv(CSV_OUT)